# Introduction
Neural rendering is an emerging but powerfull technique for photo-realistic novel view synthesis from 2D images. Its applications span from object digitalization, 3D assets generation to Virtual Reality and Augmented Reality. Creating 3D assets, in general, requires expertise in various fields that is not easily accessible to individuals. However, with this techology, one can build their own AR assets from the images taken from their phones.

Specifically, in this guide, we will guide you through the process of building your own AR assets which is fully compatible with SnapML and can be running on devices in real-time, strating from training the models to deploying the lens.


# Setup the AWS
Follow this [tutorial](https://docs.google.com/document/d/1nqCN6xZDXb_6o8rOZ0RNTydfL3ZF05fSCP03xlUJR-I/edit#heading=h.jfsq857ofbwv) on how to setup AWS for training.

# Setup the Environment
There are two major dependencies to install: [NGP_PL](https://github.com/kwea123/ngp_pl) and [MobileR2L](https://snap-research.github.io/MobileR2L/). `NGP_PL` is a Pytorch implementation of [Instant NGP](https://nvlabs.github.io/instant-ngp/) which serves as the teacher of `MobileR2L`. We will walk you through each of the installation below.



#### MobileR2L
Create a `Conda` environment and install the dependencies as below. We use `torch 2.0.1+cu11.7` as default version but this codebase is also compatiable with `torch 1.13.1+cu116`(training is slower!).

In [ ]:
# get the code
!git clone https://github.com/snap-research/MobileR2L

%cd MobileR2L

# create conda env
# download conda here: https://docs.conda.io/en/main/miniconda.html
!conda create -n r2l python==3.9
!conda activate r2l
!conda install pip

# install dependencies.
!pip install torch torchvision torchaudio
!pip install -r requirements.txt

# deactivate the env for now
!conda deactivate



#### NGP_PL
Let's first clone the repo and create a `Conda` environment.

In [ ]:
# change to the teacher directory
%cd model/teacher/ngp_pl

# create the conda env
!conda create -n ngp_pl python==3.9
!conda activate ngp_pl
!conda install pip

Next, let's install the dependencies for the `NGP_PL`.

In [ ]:
# install torch with cuda 116
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

# install tiny-cuda-nn
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

# install torch scatter
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+${cu116}.html

# ---install apex---
!git clone https://github.com/NVIDIA/apex
!cd apex
# denpency for apex
!pip install packaging

## if pip >= 23.1 (ref: https://pip.pypa.io/en/stable/news/#v23-1) which supports multiple `--config-settings` with the same key...
!pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cpp_ext" --config-settings "--build-option=--cuda_ext" ./
## otherwise
# !pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --global-option="--cpp_ext" --global-option="--cuda_ext" ./
# ---end installing apex---


%cd ../
# install other requirements
!pip install -r requirements.txt

# build
!pip install models/csrc/

# go to root
%cd ../../../


# Dataset
There are two major types of datasets for static NeRF: object-centric and forward-facing scenes. In this guide, we will be using object-centric as the example. Specifically, we use `Lego` scene from `NeRF Synthetic Dataset` [here](https://drive.google.com/drive/folders/1JDdLGDruGNXWnM1eqY1FNL9PlStjaKWi).

In [ ]:
# get the data
!mkdir -p dataset
%cd dataset
# it downloads Lego and Fern
!wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/nerf_example_data.zip
!unzip nerf_example_data.zip
%cd ..

# Training the Teacher
Now we have the data ready and we can start to train the teacher model. First, let's go to the teacher directory. Then run the train script. Once the training finished, the checkpoint will be save to `ckpts/nerf/Lego`.

In [ ]:
%cd model/teacher/ngp_pl

!export ROOT_DIR=../../../dataset/nerf_synthetic/
!python3 train.py \
     --root_dir $ROOT_DIR/lego \
     --exp_name Lego\
     --num_epochs 30 --batch_size 16384 --lr 2e-2 --eval_lpips --num_gpu 4


Once we have the teacher trained(checkpoints saved already), we can start to generate the pseudo data for `MobileR2L`. Depending your disk storage, the number of pseudo images could range from 2,000 to 10,000(performance varies!). Here, we set the number to 3000.

In [ ]:
!python3 train.py \
    --root_dir $ROOT_DIR/lego \
    --exp_name Lego_Pseudo  \
    --save_pseudo_data \
    --n_pseudo_data 3000 --weight_path ckpts/nerf/Lego/epoch=29_slim.ckpt \
    --save_pseudo_path Pseudo/lego --num_gpu 1


# Training the MobileR2L
We suggest to use ternimal with [tmux](https://github.com/tmux/tmux/wiki) instead of running it in the notebook as it takes longer to converge. Notebook might be interruppted in the middle of training. To use the terminal, simply copy the command and run it in therminal(without % and !).

In [ ]:
# change to root directory
%cd ../../../

# use r2l env
!conda activate r2l

# install the tmux
!sudo apt install tmux
# create new session
!tmux new -s lego

# run the bash
!sh benchmarking_nerf.sh



The model will be running a day or two depending on you GPUs. When the model converges, it will automatically export the onnx files to the `Experiment/Lego_**` folder. There should be three onnx files: `sampler`, `embedder` and `model`.

Alternatively, you can export the onnx manully by running the flowing script with `--ckpt_dir` replaced by the trained model:

In [ ]:
!python3 -m torch.distributed.launch --nproc_per_node=1 --use_env main.py \
      --project_name Lego \
      --dataset_type Blender \
      --pseudo_dir model/teacher/ngp_pl/Pseudo/lego \
      --root_dir dataset/nerf_synthetic \
      --export_onnx \
      --input_height 100 \
      --input_width 100 \
      --output_height 800 \
      --output_width 800 \
      --scene lego \
      --ckpt_dir Experiments/Lego-**/weights/best_ckpt.tar



#  Lens Projects

We provide two lens projects: one supports 360 degree rotation with fingers([here](https://drive.google.com/file/d/1GpbLVZ4n0HMk72im4dK4EgrjygVADTHN/view?usp=drive_link)); The other one adds surface tracking feature([here](https://drive.google.com/file/d/1FMXMhTyIRS-J_wMM7Pd7fts9vBRzq4Ay/view?usp=drive_link)). Both of the projects include some pre-built models that you can directly scan the snapcodes and run on your phones(iPhone 13 or higher is recommended).

If you wish to use your own trained model in the lens, simply import the `*_SnapGELU.onnx` to Lens Studio(need to scale the Channel 0-3 to 255.0) and 
1. for `360 degree rotation lens`: swap out the onnx file in the **first** `MLComponent` under `camera` in the left pannel of LS .
2. for `sufrace tracking lens`: swap out the onnx file in `NeRF MLComponent`.

> Figure: 360 degree rotation lens
> ![](ml-comp.png)